# ***MÓDULO LOGGING***

*Nombre:* Alegria Farinango 

*Fecha:* 22/01/2016

*Objetivo*

Conocer y utilizar el módulo de logging en Python. 

*Indicaciones*

* Investigue sobre el módulo de logging.

* Cree un ejemplo que: realice logging en consola, en un archivo, en un mensaje de Telegram. 

* Modifique el logging para mostrar el nombre del archivo, incluir la fecha y la hora, cambiar de color, etc. 

In [ ]:
import logging
import sys
from logging.handlers import RotatingFileHandler
from datetime import datetime
import requests
import os

class ColoredFormatter(logging.Formatter):
    """Formatter personalizado que añade colores a los logs en consola"""
    
    # Códigos ANSI para colores
    COLORS = {
        'DEBUG': '\033[36m',      # Cyan
        'INFO': '\033[32m',       # Verde
        'WARNING': '\033[33m',    # Amarillo
        'ERROR': '\033[31m',      # Rojo
        'CRITICAL': '\033[35m',   # Magenta
    }
    RESET = '\033[0m'
    
    def format(self, record):
        log_color = self.COLORS.get(record.levelname, self.RESET)
        record.levelname = f"{log_color}{record.levelname}{self.RESET}"
        return super().format(record)


class TelegramHandler(logging.Handler):
    """Handler personalizado para enviar logs a Telegram"""
    
    def __init__(self, bot_token, chat_id, level=logging.WARNING):
        super().__init__(level)
        self.bot_token = bot_token
        self.chat_id = chat_id
        self.url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    
    def emit(self, record):
        """Envía el log a Telegram"""
        try:
            log_entry = self.format(record)
            message = f" *Log Notification*\n\n```\n{log_entry}\n```"
            
            payload = {
                'chat_id': self.chat_id,
                'text': message,
                'parse_mode': 'Markdown'
            }
            
            requests.post(self.url, data=payload, timeout=5)
        except Exception as e:
            # Evitar recursión infinita si hay un error al enviar
            print(f"Error enviando log a Telegram: {e}")


def configurar_logger(nombre='mi_aplicacion', 
                      telegram_token=None, 
                      telegram_chat_id=None):
    """
    Configura un logger completo con handlers para consola, archivo y Telegram
    
    Args:
        nombre: Nombre del logger
        telegram_token: Token del bot de Telegram (opcional)
        telegram_chat_id: ID del chat de Telegram (opcional)
    
    Returns:
        Logger configurado
    """
    logger = logging.getLogger(nombre)
    logger.setLevel(logging.DEBUG)
    
    # Evitar duplicación de handlers si se llama múltiples veces
    if logger.handlers:
        logger.handlers.clear()
    
    # ===== 1. HANDLER DE CONSOLA CON COLORES =====
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.DEBUG)
    
    # Formato con colores para consola
    console_format = ColoredFormatter(
        fmt='%(asctime)s | %(levelname)-8s | %(filename)s:%(lineno)d | %(funcName)s() | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    console_handler.setFormatter(console_format)
    logger.addHandler(console_handler)
    
    # ===== 2. HANDLER DE ARCHIVO CON ROTACIÓN =====
    # Crear directorio de logs si no existe
    os.makedirs('logs', exist_ok=True)
    
    file_handler = RotatingFileHandler(
        filename='logs/aplicacion.log',
        maxBytes=5*1024*1024,  # 5 MB
        backupCount=5,          # Mantener 5 archivos de respaldo
        encoding='utf-8'
    )
    file_handler.setLevel(logging.INFO)
    
    # Formato detallado para archivo
    file_format = logging.Formatter(
        fmt='%(asctime)s | %(levelname)-8s | %(name)s | %(filename)s:%(lineno)d | '
            '%(funcName)s() | %(process)d | %(thread)d | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    file_handler.setFormatter(file_format)
    logger.addHandler(file_handler)
    
    # ===== 3. HANDLER DE TELEGRAM =====
    if telegram_token and telegram_chat_id:
        telegram_handler = TelegramHandler(
            bot_token=telegram_token,
            chat_id=telegram_chat_id,
            level=logging.ERROR  # Solo errores y críticos a Telegram
        )
        
        telegram_format = logging.Formatter(
            fmt='[%(levelname)s] %(asctime)s\n'
                'Archivo: %(filename)s:%(lineno)d\n'
                'Función: %(funcName)s()\n'
                'Mensaje: %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        telegram_handler.setFormatter(telegram_format)
        logger.addHandler(telegram_handler)
    
    return logger


def ejemplo_uso():
    """Función de ejemplo que demuestra el uso del logger"""
    
    # Configurar logger
    # Para usar Telegram, descomenta y añade tus credenciales:
    # logger = configurar_logger(
    #     nombre='demo_app',
    #     telegram_token='TU_BOT_TOKEN',
    #     telegram_chat_id='TU_CHAT_ID'
    # )
    
    logger = configurar_logger(nombre='demo_app')
    
    logger.debug("Iniciando la aplicación - esto es un mensaje de depuración")
    logger.info("Aplicación iniciada correctamente")
    
    # Simulación de operaciones
    try:
        logger.info("Procesando datos del usuario")
        resultado = procesar_datos(42)
        logger.info(f"Datos procesados exitosamente: {resultado}")
    except ValueError as e:
        logger.error(f"Error al procesar datos: {e}", exc_info=True)
    
    # Simular una advertencia
    logger.warning("Memoria RAM al 85% de capacidad")
    
    # Simular error crítico
    try:
        operacion_critica()
    except Exception as e:
        logger.critical(f"Error crítico en operación: {e}", exc_info=True)


def procesar_datos(valor):
    """Función de ejemplo que procesa datos"""
    logger = logging.getLogger('demo_app')
    logger.debug(f"Procesando valor: {valor}")
    
    if valor < 0:
        raise ValueError("El valor no puede ser negativo")
    
    return valor * 2


def operacion_critica():
    """Función que simula un error crítico"""
    logger = logging.getLogger('demo_app')
    logger.debug("Ejecutando operación crítica")
    
    # Simular un error
    raise RuntimeError("Fallo en la conexión a la base de datos")


if __name__ == "__main__":
    print("=" * 80)
    print("DEMO DEL SISTEMA DE LOGGING")
    print("=" * 80)
    print()
    
    ejemplo_uso()
    
    print("\n" + "=" * 80)
    print("Revisa el archivo 'logs/aplicacion.log' para ver los logs guardados")
    print("=" * 80)